# Transfer-Learning-Using-Keras

###Classification of glass vs platstic bottles using keras and python3.6+

## Installing requirements

```bash
pip install tensorflow
pip install keras
pip3 install pillow
```

## Example: train the model and classify an image

```python
'''we have only two functions in the code (train and classify) to make it easy to use'''
>>> train() # will train the model and save it to 'My_resNet50_weights.h5' by default
>>> classify('path/to/image') # just replace the path with one image of bottle you want to classify
```

In [1]:
'''
I will use transfer learning so that I can only take about 80 plastic bottles
and 80 glass bottles on google image to get a descent result (above 90% accuray)
at classifying if the bottle is made of glass or plastic.
Also I will use data augmentation during preprocessing.
The model takes a few minutes to train on cpu.


the dataset is on dropbox

dataset dropbox link:
https://www.dropbox.com/s/40274pr5d9xgct6/dataset.zip?dl=0

'''

import keras
from keras import models, layers
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np
from PIL import Image


def train():
    '''
    This function trains a neural network to predict if the bottle
    on the input image is made of glass or plastic.
    It will download the dataset for training on dropbox.
    A 'My_resNet50_weights.h5' file will be created,
    it contains our model and weights.
    '''
    
    # download and prepare the dataset for training
    !wget "https://www.dropbox.com/s/40274pr5d9xgct6/dataset.zip?dl=1" -O "dataset.zip" -c
    !unzip dataset.zip

    
    # adapt the path to the dataset location if needed
    TRAIN_DIR = "./dataset/train"
    VALID_DIR = "./dataset/validation"
    # standards parameters for resnet
    HEIGHT = 224
    WIDTH = 224
    # arbitrary batch size
    BATCH_SIZE = 8

    # I start from resnet50 pretrained model
    # but I do not take its last layer
    base_model = ResNet50(weights='imagenet',
                     include_top=False, # discard the last layer, we will train our own
                     input_shape=(HEIGHT, WIDTH, 3))

    # data preprocessing and augmentation
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input, # normalize images with imagenet stats
        horizontal_flip=True, # a bottle can be in any position so we should rotate it
        vertical_flip=True
        )

    # creating data generators to feed to the model
    train_generator = datagen.flow_from_directory(directory=TRAIN_DIR, 
                                                        target_size=(HEIGHT, WIDTH),
                                                        color_mode="rgb",
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical",
                                                        shuffle=True,
                                                        seed=42
                                                       )

    valid_generator = datagen.flow_from_directory(directory=VALID_DIR, 
                                                        target_size=(HEIGHT, WIDTH),
                                                        color_mode="rgb",
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical",
                                                        shuffle=True,
                                                        seed=42
                                                       )

    # freezing the pretrained weights from resnet50,
    # we do not need to train the whole model
    for layer in base_model.layers:
        layer.trainable = False
    # if we wanted to improve accuracy by a few percents, we could unfreeze and
    # train the whole model
    # we sould do this with discriminative learning rates so that the early layers
    # will train less as they are already good at doing their role (detecting basic shapes)

    # taking the previously modified pretrained resnet50 model
    model = base_model.output


    # we flatten the output from the base_model
    # because we want to pass it to our fully connected layer (classifier)
    model = layers.Flatten()(model)


    # adding our own layer for training
    # here we could add droupout if we were overfitting
    model = layers.Dense(1024, activation='relu')(model)
    model = layers.Dense(512, activation='relu')(model)

    # softmax activation on the last layer,
    # as we want a probability for classification
    predictions = layers.Dense(2, activation='softmax')(model) 

    # define our prediction model
    prediction_model = models.Model(inputs=base_model.input, outputs=predictions)


    # the total of images we are using for training and validation
    num_train_images = 66
    num_valid_images = 15

    # how many times we will present the dataset to the model
    NUM_EPOCHS = 8
    
    # our optinizer
    adam = Adam(lr=1e-4)

    # configure the model for training
    prediction_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

    # path to save our model
    filepath="./My_resNet50_weights.h5"

    # save the model and weights when needed
    # (it will save the best results of all epochs)
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_best_only=True,
                                                 monitor='acc', # monitor accuracy
                                                 verbose=1)

    # trains the model and stores training loss values and metrics values
    # at successive epochs, as well as validation loss and metrics values
    history = prediction_model.fit_generator(generator=train_generator,
                                             epochs=NUM_EPOCHS,
                                             steps_per_epoch=num_train_images // BATCH_SIZE,
                                             workers=8,
                                             validation_data=valid_generator,
                                             validation_steps=num_valid_images,
                                             callbacks=[checkpoint]
                                             )

Using TensorFlow backend.


In [0]:
def classify(image):
    '''
    returns: -> 0 if it is a glass bottle in the image
             -> 1 if it is a plastic bottle in the image
    
    image: a string which is an image path
    '''
    
    # Load the model we previously trained
    # (if needed comment/uncomment the next line)
    prediction_model = models.load_model('My_resNet50_weights.h5')

    # read the image
    image = Image.open(image)
    
    # adapt the image for the neural net
    image = image.convert('RGB')
    image = image.resize((224, 224))
    image = np.array(image, dtype=np.float64)
    
    # add a dimension to input to the network
    image = image[None, ...]
    
    # add resnet stats as the model was trained with those stats
    image /= 255.
    mean = [0.485, 0.456, 0.406] # Here it's ImageNet statistics
    std = [0.229, 0.224, 0.225]

    # normalize image with resnet stats, considering an ordering
    # (batch, height, width, channel): the same as image.shape
    for i in range(3): 
        image[0][:, :, i] -= mean[i]
        image[0][:, :, i] /= std[i]

    # result equals 0 if it is glass, and 1 for plastic
    # we take argmin because the results from keras are [plastic, glass]
    result = np.argmin(prediction_model.predict(x=image))
    
    return result
    

In [4]:
train()

--2019-04-02 18:16:41--  https://www.dropbox.com/s/40274pr5d9xgct6/dataset.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/40274pr5d9xgct6/dataset.zip [following]
--2019-04-02 18:16:41--  https://www.dropbox.com/s/dl/40274pr5d9xgct6/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1cd19adef7ed7f462b4ff850e4.dl.dropboxusercontent.com/cd/0/get/AeRjrhH1vloQQeDL3B5aVFH7h0Jv2Wk2O3zH5JSvqkdTTNntEGoJT3rG8TpOz2xfFoHBIMf9m4SQhgpo0jYCJqxVuGtMWxOZ6gWcnF1Jl5KMnS7L3s6jbRx3_RT8uUgtS30/file?dl=1# [following]
--2019-04-02 18:16:41--  https://uc1cd19adef7ed7f462b4ff850e4.dl.dropboxusercontent.com/cd/0/get/AeRjrhH1vloQQeDL3B5aVFH7h0Jv2Wk2O3zH5JSvqkdTTNntEGoJT3rG8TpOz2xfFoHBIMf9m4SQhgpo0jYCJqxVuGtMWxOZ6gWcnF1Jl5KMnS7L3s6

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 132 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Epoch 1/8
8/8 [==============================] - 79s 10s/step - loss: 3.2286 - acc: 0.5625 - val_loss: 4.6815 - val_acc: 0.6053

Epoch 00001: acc improved from -inf to 0.56250, saving model to ./My_resNet50_weights.h5
Epoch 2/8
8/8 [==============================] - 69s 9s/step - loss: 2.6472 - acc: 0.6562 - val_loss: 2.1960 - val_acc: 0.7768

Epoch 00002: acc improved from 0.56250 to 0.65625, saving model to ./My_resNet50_weights.h5
Epoch 3/8
8/8 [==============================] - 68s 8s/step - loss: 2.4434 - acc: 0.7657 - val_loss: 2.0945 - val_acc: 0.7857

Epoch 00003: acc improved from 0.65625 to 0.76667, saving model to ./My_resNet50_weights.h5
Epoch 4/8
8/8 [==============================] - 68s 8s/step - loss: 1.4668 - acc: 0.8281 - val_loss: 1.5750 - val_acc: 0.8393

Epoch 00004: acc improved from 0.76667 to 0.82812, saving model to ./My_resNet50_weights.h5
Epoch 5/8
8/8 [==========================

In [9]:
# enter your image path here
classify('./dataset/validation/verre/-bouteille-en-verre-all-round-100ml-18mm-brun.jpg')

0